In [7]:
import numpy as np
from multiprocessing import Pool
import time

# Nueva función de reducción
def sum_multiprocessing(A, ini, fin):
    """Suma los elementos del array A desde 'ini' hasta 'fin'."""
    return np.sum(A[ini:fin])

# Tamaño del array y creación del mismo
value = 5 * 10**6
X = np.random.rand(value)

# Definición de los segmentos
segments_1 = [(X, 0, value)]
segments_2 = [(X, 0, int(value/2)), (X, int(value/2), value)]
segments_4 = [
    (X, 0, int(value/4)), 
    (X, int(value/4), int(value/2)), 
    (X, int(value/2), int(3*value/4)), 
    (X, int(3*value/4), value)
]

# Mediciones
start_time = time.perf_counter()
with Pool(processes=1) as pool:
    result_1 = pool.starmap(sum_multiprocessing, segments_1)
end_time = time.perf_counter()
print("Tiempo con 1 proceso:", end_time - start_time, "s")

start_time = time.perf_counter()
with Pool(processes=2) as pool:
    result_2 = pool.starmap(sum_multiprocessing, segments_2)
end_time = time.perf_counter()
print("Tiempo con 2 procesos:", end_time - start_time, "s")

start_time = time.perf_counter()
with Pool(processes=4) as pool:
    result_4 = pool.starmap(sum_multiprocessing, segments_4)
end_time = time.perf_counter()
print("Tiempo con 4 procesos:", end_time - start_time, "s")


Tiempo con 1 proceso: 0.04704229161143303 s
Tiempo con 2 procesos: 0.07912963442504406 s
Tiempo con 4 procesos: 0.14585491456091404 s


In [8]:
import cupy as cp
import numpy as np
import time

# Crear el array en la CPU
value = 5 * 10**6
X_cpu = np.random.rand(value)

# Mediciones
start_time = time.perf_counter()

# Copiar el array a la GPU
X_gpu = cp.array(X_cpu)

# Sumar los elementos en la GPU
result_gpu = cp.sum(X_gpu)

# Copiar el resultado a la CPU
result_cpu = result_gpu.get()

end_time = time.perf_counter()

print("Resultado obtenido en la GPU y copiado a la CPU:", result_cpu)
print("Tiempo total (CPU->GPU, operación en GPU, GPU->CPU):", end_time - start_time, "s")


Resultado obtenido en la GPU y copiado a la CPU: 2499871.8847916257
Tiempo total (CPU->GPU, operación en GPU, GPU->CPU): 0.019719351083040237 s


In [9]:
from numba import cuda
import numpy as np
import time

@cuda.jit
def sum_reduce_kernel(data, result):
    """
    Suma los elementos del array `data` en paralelo y almacena el resultado en `result`.
    """
    idx = cuda.grid(1)
    if idx < data.size:
        cuda.atomic.add(result, 0, data[idx])

# Crear datos en la CPU
value = 5 * 10**6
X_cpu = np.random.rand(value).astype(np.float32)

# Copiar datos a la GPU
X_gpu = cuda.to_device(X_cpu)
result_gpu = cuda.device_array(1, dtype=np.float32)

# Configurar el kernel
threads_per_block = 256
blocks_per_grid = (value + (threads_per_block - 1)) // threads_per_block

# Medición
start_time = time.perf_counter()

# Ejecutar el kernel
sum_reduce_kernel[blocks_per_grid, threads_per_block](X_gpu, result_gpu)

# Copiar el resultado a la CPU
result_cpu = result_gpu.copy_to_host()

end_time = time.perf_counter()

print("Resultado de la reducción con Numba:", result_cpu[0])
print("Tiempo total con Numba:", end_time - start_time, "s")


Resultado de la reducción con Numba: 2499706.2
Tiempo total con Numba: 0.058716313913464546 s
